## Lookback Put Option.

Contribution by: [Kumarjb](https://github.com/Kumarjb)

This script breaks down the implementation of a lookback put option (floating strike) timetable. 

- It is packaged as a single method [here](./src/contracts/lookback.py).
- It has been validated against literature results [here](04a_LookbacksValidation.ipynb)
- It has been priced with different models [here](./04b_Lookbacks_Model_Prices.ipynb)

In [1]:
from datetime import timedelta

import numpy as np
import pandas as pd
from pyarrow import RecordBatch as rb
from qablet_contracts.timetable import TS_EVENT_SCHEMA

In [2]:
ticker = "SPX"
start_date = pd.to_datetime("2005-09-14")
T = 0.2
num_points = 4

find fixing dates, including the start date, but not the maturity date. The minimum of the spots on these dates will be the floating strike.

In [3]:
days_to_maturity = T * 365.25
maturity = start_date + timedelta(days=days_to_maturity)

fix_dates = pd.date_range(
    start=start_date,
    end=maturity,
    periods=num_points + 1,
    inclusive="left",
)

initialize the floating strike.

In [4]:
events = [
    {
        "track": "",
        "time": fix_dates[0],
        "op": None,
        "quantity": 0,
        "unit": "INIT",
    }
]

update the floating strike at each subsequent fixing date.

In [5]:
for fixing_time in fix_dates[1:]:
    events.append(
        {
            "track": "",
            "time": fixing_time,
            "op": None,
            "quantity": 0,
            "unit": "UPDATE",
        }
    )

the final payoff.

In [6]:
events.append(
    {
        "track": "",
        "time": maturity,
        "op": "+",
        "quantity": 1,
        "unit": "LOOKBACK",
    }
)

now define the init, update and the payoff functions.

In [7]:
# Defining fixed strike look-back put payoff function
def lookback_put_pay_fn(inputs):
    [ticker, s_max] = inputs
    return [np.maximum(s_max - ticker, 0)]


events_table = rb.from_pylist(events, schema=TS_EVENT_SCHEMA)
timetable = {
    "events": events_table,
    "expressions": {
        "LOOKBACK": {
            "type": "phrase",
            "inp": [ticker, "MAX_SPOT"],
            "fn": lookback_put_pay_fn,
        },
        "UPDATE": {
            "type": "snapper",
            "inp": [ticker, "MAX_SPOT"],
            "fn": lambda inputs: [np.maximum(inputs[0], inputs[1])],
            "out": ["MAX_SPOT"],
        },
        "INIT": {
            "type": "snapper",
            "inp": [ticker],
            "fn": lambda inputs: inputs,
            "out": ["MAX_SPOT"],
        },
    },
}

In [8]:
print(timetable["events"].to_pandas())

                       time   op  quantity      unit track
0 2005-09-14 00:00:00+00:00  NaN       0.0      INIT      
1 2005-10-02 06:18:00+00:00  NaN       0.0    UPDATE      
2 2005-10-20 12:36:00+00:00  NaN       0.0    UPDATE      
3 2005-11-07 18:54:00+00:00  NaN       0.0    UPDATE      
4 2005-11-26 01:12:00+00:00    +       1.0  LOOKBACK      
